#Intelligent News Summarization and Analysis system

###Description:
- The implementation integrates NewsAPI, OpenAI GPT API (via LangChain for simplified prompt management), FastAPI for API handling, and other essential libraries for processing, caching, and managing the pipeline.

###Developed By : Naveen Kumar

##Install the necessary libraries

In [ ]:
!pip install newsapi-python langchain openai fastapi uvicorn beautifulsoup4 requests
!pip install -q fastapi-cache2 pytest docker

In [ ]:
import os
import requests
from newsapi import NewsApiClient
from fastapi import FastAPI, HTTPException, Query
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from bs4 import BeautifulSoup
from typing import List, Dict, Any
from fastapi_cache import caches, close_caches
from fastapi_cache.backends.inmemory import InMemoryCacheBackend
import uvicorn
import re
from datetime import datetime
import asyncio

##API Integration

###Initialize NewsAPI and OpenAI API clients

In [ ]:
NEWS_API_KEY = "YOUR_NEWSAPI_KEY"
OPENAI_API_KEY = "YOUR_OPENAI_KEY"
newsapi = NewsApiClient(api_key=NEWS_API_KEY)
llm = OpenAI(api_key=OPENAI_API_KEY)

#### Initialize FastAPI and caching

In [ ]:
app = FastAPI()
caches.set(InMemoryCacheBackend())

##Data Collection

###Fetching news articles

In [ ]:
# Define function for fetching news articles
async def fetch_news(query: str, page_size: int = 5) -> List[Dict[str, Any]]:
    try:
        articles = newsapi.get_everything(q=query, language="en", page_size=page_size)
        return articles["articles"]
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

###Article processing

In [ ]:
# Define function for article processing
def process_article(article: Dict[str, Any]) -> Dict[str, Any]:
    content = article.get("content", "")
    cleaned_content = re.sub(r'\W+', ' ', content)
    article_data = {
        "title": article["title"],
        "date": article["publishedAt"],
        "content": cleaned_content,
        "source": article["source"]["name"]
    }
    return article_data

###Prompt templates

In [ ]:
# Define prompt templates for each LLM task
summary_prompt = PromptTemplate("Summarize the following article: {content}")

In [ ]:
key_points_prompt = PromptTemplate("Extract key points from this article: {content}")

In [ ]:
sentiment_prompt = PromptTemplate("Analyze the sentiment of the article: {content}")

In [ ]:
topic_prompt = PromptTemplate("Classify the main topic of the article: {content}")

###LLM Integration

####Interaction with LLM

In [ ]:
# Function to interact with LLM for tasks
async def analyze_article_with_llm(content: str) -> Dict[str, str]:
    tasks = {
        "summary": summary_prompt.generate(content=content),
        "key_points": key_points_prompt.generate(content=content),
        "sentiment": sentiment_prompt.generate(content=content),
        "topic": topic_prompt.generate(content=content)
    }
    results = await asyncio.gather(
        llm(tasks["summary"]),
        llm(tasks["key_points"]),
        llm(tasks["sentiment"]),
        llm(tasks["topic"])
    )
    return {
        "summary": results[0],
        "key_points": results[1],
        "sentiment": results[2],
        "topic": results[3]
    }

###API Development

####Asynchronous endpoints

In [ ]:
# Asynchronous endpoint for fetching and analyzing articles
@app.get("/news/")
async def get_news(query: str = Query(..., min_length=2)):
    cached_data = await caches.get("news_cache", key=query)
    if cached_data:
        return cached_data

    articles = await fetch_news(query)
    if not articles:
        raise HTTPException(status_code=404, detail="No articles found.")

    processed_articles = []
    for article in articles:
        data = process_article(article)
        analysis = await analyze_article_with_llm(data["content"])
        processed_articles.append({**data, **analysis})

    await caches.set("news_cache", key=query, value=processed_articles, ttl=300)
    return processed_articles


In [ ]:
# Endpoint to retrieve trending topics
@app.get("/trending/")
async def get_trending_topics():
    # Placeholder for actual trending topic extraction
    return {"topics": ["technology", "business", "politics"]}


###FastAPI Caching

In [ ]:
# Caching setup for FastAPI
@app.on_event("startup")
async def startup_event():
    await caches.init()

@app.on_event("shutdown")
async def shutdown_event():
    await close_caches()


###Docker Setup

In [ ]:
# Dockerfile configuration
DOCKERFILE = """
# Use official Python image as a base image
FROM python:3.9

# Set the working directory
WORKDIR /app

# Copy the requirements file and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code
COPY . .

# Expose port 8000 and run the FastAPI app
EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""


###Unit Testing

In [ ]:
# Unit tests
def test_process_article():
    article = {"title": "Test", "publishedAt": "2023-01-01", "content": "Some content", "source": {"name": "Test Source"}}
    processed = process_article(article)
    assert processed["title"] == "Test"
    assert processed["date"] == "2023-01-01"


In [ ]:
# Run unit tests with pytest
!echo "test_process_article()" > test_script.py
!pytest test_script.py > result.log

###Saving the Output

In [ ]:
# Dockerfile output to save to file
with open("Dockerfile", "w") as f:
    f.write(DOCKERFILE)

###Run the FastAPI server locally

In [ ]:
uvicorn main:app --reload --host 0.0.0.0 --port 8000